# Avaliação de Classificadores KNN com K-Fold Cross-Validation - Dataset Vinhos (UCI)

## 1. Leitura dos Dados

In [2]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

## 2. Pré-processamento dos Dados

In [3]:
X = vinhos.drop(columns=['classe'])
y = vinhos['classe']

## 3. Configuração do Experimento

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Função para realizar um experimento com um dado random_state
def experimento(X, y, random_state, k_values):
    # Embaralhar os dados
    np.random.seed(random_state)
    shuffled_indices = np.random.permutation(len(X))
    X_shuffled = X.iloc[shuffled_indices]
    y_shuffled = y.iloc[shuffled_indices]

    # Divisão 70% treino e 30% teste
    train_size = int(0.7 * len(X))
    X_train, X_test = X_shuffled[:train_size], X_shuffled[train_size:]
    y_train, y_test = y_shuffled[:train_size], y_shuffled[train_size:]

    resultados = []

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)

        knn.fit(X_train, y_train)

        y_pred = knn.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        resultados.append({'random_state': random_state, 'k': k, 'accuracy': accuracy})

        print(f"Random State {random_state} | k={k} | Acurácia: {accuracy:.4f}")

    return resultados

random_states = [42, 17, 24]
k_values = [3, 5]

arrayTodosResultados = []
for random_state in random_states:
    arrayTodosResultados.extend(experimento(X, y, random_state, k_values))

import pandas as pd
resultados_df = pd.DataFrame(arrayTodosResultados)

melhor_resultado = resultados_df.loc[resultados_df['accuracy'].idxmax()]
print("\nMelhor Configuração Geral:")
print(f"Random State: {melhor_resultado['random_state']}, k: {melhor_resultado['k']}, Acurácia: {melhor_resultado['accuracy']:.4f}")


Random State 42 | k=3 | Acurácia: 0.6667
Random State 42 | k=5 | Acurácia: 0.5741
Random State 17 | k=3 | Acurácia: 0.8148
Random State 17 | k=5 | Acurácia: 0.8148
Random State 24 | k=3 | Acurácia: 0.6852
Random State 24 | k=5 | Acurácia: 0.6481

Melhor Configuração Geral:
Random State: 17.0, k: 3.0, Acurácia: 0.8148


## 4. Estrutura de Cross-Validation com KFold

In [5]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

k_values = [3, 5]
random_states = [42, 17, 24]
results = []

for random_state in random_states:
    print(f"\nRandom State: {random_state}")

    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    for k in k_values:
        fold_accuracies = []

        for train_index, test_index in kf.split(X):

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)

            y_pred = knn.predict(X_test)

            accuracy = accuracy_score(y_test, y_pred)
            fold_accuracies.append(accuracy)

        # Calcular a média e o desvio padrão das acurácias para este valor de k
        mean_accuracy = np.mean(fold_accuracies)
        std_accuracy = np.std(fold_accuracies)

        # Registrar os resultados
        results.append({
            'random_state': random_state,
            'k': k,
            'mean_accuracy': mean_accuracy,
            'std_accuracy': std_accuracy
        })

        print(f"k={k} | Média da Acurácia: {mean_accuracy:.4f} | Desvio Padrão: {std_accuracy:.4f}")

results_df = pd.DataFrame(results)

# Exibir resultados e melhor configuração geral
print("\nResultados Consolidados:")
print(results_df)

best_result = results_df.loc[results_df['mean_accuracy'].idxmax()]
print("\nMelhor Configuração Geral:")
print(f"Random State: {best_result['random_state']}, k: {best_result['k']}, "
      f"Média da Acurácia: {best_result['mean_accuracy']:.4f}, "
      f"Desvio Padrão: {best_result['std_accuracy']:.4f}")



Random State: 42
k=3 | Média da Acurácia: 0.7036 | Desvio Padrão: 0.1392
k=5 | Média da Acurácia: 0.6641 | Desvio Padrão: 0.1001

Random State: 17
k=3 | Média da Acurácia: 0.6987 | Desvio Padrão: 0.1040
k=5 | Média da Acurácia: 0.6876 | Desvio Padrão: 0.1121

Random State: 24
k=3 | Média da Acurácia: 0.7127 | Desvio Padrão: 0.0968
k=5 | Média da Acurácia: 0.7127 | Desvio Padrão: 0.0940

Resultados Consolidados:
   random_state  k  mean_accuracy  std_accuracy
0            42  3       0.703595      0.139187
1            42  5       0.664052      0.100075
2            17  3       0.698693      0.104030
3            17  5       0.687582      0.112118
4            24  3       0.712745      0.096816
5            24  5       0.712745      0.093973

Melhor Configuração Geral:
Random State: 24.0, k: 3.0, Média da Acurácia: 0.7127, Desvio Padrão: 0.0968
